In [ ]:
pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=c046758ed4b5255d69109fcd4cb641f254f51e86892fc5ada69dc08a3bea30ce
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [ ]:
pip install requests

In [ ]:
pip install bs4

In [ ]:
!pip install -U -q PyDrive

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import kurtosis, skew
from tabulate import tabulate
import datetime as dt#get current date
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import requests
from bs4 import BeautifulSoup
import matplotlib.ticker as mtick
from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def stock_return(Ticker,Start_T, End_T, T_interval):
  if T_interval == '1y':
    Stock_price = yf.download(Ticker,start=Start_T,end=End_T,interval = '1mo')['Adj Close'].dropna()
    Stock_price_year = []
    date_year = pd.date_range(start=Start_T,end=End_T, freq='Y')
    for i in range(len(Stock_price)):
      if i % 12 == 0:
        Stock_price_year.append(Stock_price[i])
    s1 = pd.Series(date_year,name='Date')
    s2=  pd.Series(Stock_price_year[:len(date_year)],name='year_stock_price')
    Ret_y = s2.pct_change()[1:].dropna()
    Ret_y_d = pd.concat([s1, Ret_y],axis=1).dropna()
    Ret_y_d = Ret_y_d.set_index('Date')
    return(Ret_y_d)
  else:
    Stock_price = yf.download(Ticker,start=Start_T,end=End_T,interval = T_interval)['Adj Close'].dropna()
    Ret = Stock_price.pct_change()[1:].dropna()
    return Ret

In [ ]:
previousday = dt.date.today()-dt.timedelta(days=1)# get the date of previous day
Default_end = previousday.strftime("%Y-%m-%d") # change the format of the default datetime 
years_ago = dt.date.today()-dt.timedelta(days=1852)
Default_start = years_ago.strftime("%Y-%m-%d")

Ticker = input('Please enter ticker(Default = JNJ)-->') or 'JNJ'
Start = input('Enter start date(' + 'Default = 5 Years ago, ' + Default_start + ')-->') or Default_start
End = input('Enter end date(Default = Yesterday, ' + Default_end + ')-->') or Default_end
T_interval = input('Enter date interval(1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo Default = 1mo)-->') or '1mo'

Ret = stock_return(Ticker,Start, End, T_interval)
Ret1 = stock_return('SNP',Start, End, T_interval)
#Ret.sort_index(ascending=True)
#Ret1.sort_index(ascending=True)
if T_interval != '1y':
  Ret.to_frame()
  Ret1.to_frame()
Retcon = pd.concat([Ret,Ret1], axis = 1)
Retcon.columns = [Ticker, 'SNP']
print(Retcon)

Please enter ticker(Default = JNJ)-->
Enter start date(Default = 5 Years ago, 2015-06-01)-->
Enter end date(Default = Yesterday, 2020-06-25)-->
Enter date interval(1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo Default = 1mo)-->1y
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                 JNJ       SNP
Date                          
2016-12-31  0.281621 -0.105466
2017-12-31  0.120158  0.124585
2018-12-31 -0.058406  0.283561
2019-12-31  0.178579 -0.177723


In [ ]:
ret_M = stock_return('SNP', Start,End,T_interval)
ret_a = stock_return(Ticker, Start,End, T_interval)
ret_M_arr = np.asarray(ret_M)
ret_a_arr = np.asarray(ret_a)
ret_Mc = sm.add_constant(ret_M_arr)
results = sm.OLS(ret_a_arr,ret_Mc).fit()
print('                                 Market Model')
print(results.summary2())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                                 Market Model
                 Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.650    
Dependent Variable: y                AIC:                -7.2046  
Date:               2020-06-26 03:17 BIC:                -8.4320  
No. Observations:   4                Log-Likelihood:     5.6023   
Df Model:           1                F-statistic:        6.568    
Df Residuals:       2                Prob (F-statistic): 0.124    
R-squared:          0.767            Scale:              0.0071126
--------------------------------------------------------------------
             Coef.    Std.Err.      t      P>|t|     [0.025   0.975]
--------------------------------------------------------------------
const        0.1489     0.0428    3.4806   0.0736   -0.0352   0.3329
x1          -0.5888   

/usr/local/lib/python3.6/dist-packages/statsmodels/stats/stattools.py:71: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  "samples were given." % int(n), ValueWarning)


In [ ]:
model = ols(Ticker+' ~ SNP',Retcon).fit()
anova_table = anova_lm(model)
print(anova_table)

           df    sum_sq   mean_sq         F    PR(>F)
SNP       1.0  0.046716  0.046716  6.568078  0.124457
Residual  2.0  0.014225  0.007113       NaN       NaN


In [ ]:
total_risk = round((anova_table['sum_sq'][0]+anova_table['sum_sq'][1])/(anova_table['df'][0]+anova_table['df'][1]),5)
systematic_risk = round((anova_table['sum_sq'][0])/(anova_table['df'][0]+anova_table['df'][1]),5)
unsystematic_risk = round((anova_table['sum_sq'][1])/(anova_table['df'][0]+anova_table['df'][1]),5)
print('By using the rate of return for ' + str(Ticker) + ' and the market rate of return to estimate the market model \nWe obtain total risk as ' + str(total_risk)+'\nSystematic risk as '+ str(systematic_risk) + '\nUnsystematic risk as '+ str(unsystematic_risk))

By using the rate of return for JNJ and the market rate of return to estimate the market model 
We obtain total risk as 0.02031
Systematic risk as 0.01557
Unsystematic risk as 0.00474
